# DLVC 2017
# Tutorial 3: Cifar-10 Classification (Classical Approach)

#### Dataset: Cifar-10 [https://www.cs.toronto.edu/~kriz/cifar.html]

In [ ]:
%matplotlib inline
from torchvision import datasets
from pylab import *
import pickle
from skimage import feature
from sklearn import preprocessing
import os
import matplotlib.pyplot as plt
from PIL import Image
import zipfile


if not os.path.exists('tut03-results'):
    os.makedirs('tut03-results')

In [ ]:
# Downloading dataset using pytorh-torchvision
trainSet = datasets.CIFAR10('./CIFAR10/', train=True, download=True)
testSet = datasets.CIFAR10('./CIFAR10/', train=False, download=True)

In [ ]:
print('No. of training samples: '+str(len(trainSet)))
print('No. of testing samples: '+str(len(testSet)))

In [ ]:
train_greycoHomFeat = []
train_greycoConFeat = []
train_greycoEnFeat = []
train_greycoCorrFeat = []
train_hogFeat = []
train_lbpFeat = []
train_label = np.zeros(len(trainSet))
for num in range(len(trainSet)):
    if (num+1)%10000==0:
        print(str(num+1)+'/'+str(len(trainSet)))
    img = trainSet[num][0]
    train_label[num] = trainSet[num][1]
    train_greycoHomFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='homogeneity'))
    train_greycoConFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='contrast'))
    train_greycoEnFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='energy'))
    train_greycoCorrFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='correlation'))
    train_hogFeat.append(feature.hog(array(img.convert('L')), orientations=4, pixels_per_cell=(15,15))) 
    train_lbpFeat.append(feature.local_binary_pattern(array(img.convert('L')), 5, 3)) 

In [ ]:
test_greycoHomFeat = []
test_greycoConFeat = []
test_greycoEnFeat = []
test_greycoCorrFeat = []
test_hogFeat = []
test_lbpFeat = []
test_label = np.zeros(len(testSet))
for num in range(len(testSet)):
    if (num+1)%5000==0:
        print(str(num+1)+'/'+str(len(testSet)))
    img = testSet[num][0]
    test_label[num] = testSet[num][1]
    test_greycoHomFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='homogeneity'))
    test_greycoConFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='contrast'))
    test_greycoEnFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='energy'))
    test_greycoCorrFeat.append(feature.greycoprops(feature.greycomatrix(array(img.convert('L')), [1], [np.pi/4],normed=True),prop='correlation'))                          
    test_hogFeat.append(feature.hog(array(img.convert('L')), orientations=4, pixels_per_cell=(15,15))) 
    test_lbpFeat.append(feature.local_binary_pattern(array(img.convert('L')), 5, 3)) 

In [ ]:
# Data
trainFeat = np.zeros((len(train_label),1028))
for num in range(len(train_label)):    
    trainFeat[num][:] = np.concatenate((train_greycoHomFeat[num].reshape(1,),train_greycoConFeat[num].reshape(1,),
                            train_greycoEnFeat[num].reshape(1,),train_greycoCorrFeat[num].reshape(1,),
                                        train_hogFeat[num],train_lbpFeat[num].reshape(32*32)),axis=0)

testFeat = np.zeros((len(test_label),1028))
for num in range(len(test_label)):    
    testFeat[num][:] = np.concatenate((test_greycoHomFeat[num].reshape(1,),test_greycoConFeat[num].reshape(1,),
                            test_greycoEnFeat[num].reshape(1,),test_greycoCorrFeat[num].reshape(1,),
                                        test_hogFeat[num],test_lbpFeat[num].reshape(32*32)),axis=0)
trainFeat_scaled = preprocessing.scale(trainFeat)
testFeat_scaled = preprocessing.scale(testFeat)

In [ ]:
# with open('tut03-results/trainFeat_scaled.pkl','wb') as f:
#     pickle.dump(trainFeat_scaled,f)
# with open('tut03-results/testFeat_scaled.pkl','wb') as f:
#     pickle.dump(testFeat_scaled,f)

scikitlearn [documentation](goo.gl/F1Q1Fa) for MLP classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(hidden_layer_sizes=(100,), max_iter=10, alpha=1e-4,
                    solver='sgd', verbose=True, tol=1e-4, random_state=1,
                    learning_rate_init=1e-2)
nn.fit(trainFeat_scaled, train_label)       
prediction = nn.predict(testFeat_scaled)

In [ ]:
print("Training set score: %f" % nn.score(trainFeat_scaled, train_label)) # mean accuracy
print("Test set score: %f" % nn.score(testFeat_scaled, test_label))